In [1]:
import csv
import json
import pandas as pd
from datetime import datetime
import numpy as np

outputFolders = "../data_2019_processed/test/"

fileRaw = "allData.csv"
fileNumeric = "-numeric.csv"
fileNumericNormalized = "-numeric-normalized.csv"
fileNumericClassification = "-numeric-classification.csv"
fileNumericClassificationNormalized = "-classification-numeric-normalized.csv"

In [2]:
def convert_to_decibels(value):
    if(value > 0.0):
        return 20 * (np.log(32768) - np.log(value/32768) )
        #return 20 * ( math.log(32768/value) )
        #return 20 * (np.log(value)) 
    else:
        return 0.0

def treat_data_to_0_1(value):
    if(value > 0.0):
        return 1.0
        #return 20 * ( math.log(32768/value) )
        #return 20 * (np.log(value)) 
    else:
        return 0.0
    
def transform_data(dataframe):
    # convert data and show the new info
    # activity
    dataframe['activity'] = dataframe['activity'].replace(['Unknown'], '1')
    dataframe['activity'] = dataframe['activity'].replace(['Still'], '2')
    dataframe['activity'] = dataframe['activity'].replace(['Tilting'], '3')
    dataframe['activity'] = dataframe['activity'].replace(['Exercise'], '4')
    dataframe['activity'] = dataframe['activity'].replace(['In vehicle'], '5')
    # class
    dataframe['class'] = dataframe['class'].replace(['awake'], '0')
    dataframe['class'] = dataframe['class'].replace(['asleep'], '1')
    # day_of_week
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Tuesday'], '1')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Wednesday'], '2')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Thursday'], '3')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Friday'], '4')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Saturday'], '5')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Sunday'], '6')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Monday'], '7')
    # phone_lock
    dataframe['phone_lock'] = dataframe['phone_lock'].replace(['not locked'], '0')
    dataframe['phone_lock'] = dataframe['phone_lock'].replace(['locked'], '1')
    # location
    dataframe['location'] = dataframe['location'].replace(['Other'], '1')
    dataframe['location'] = dataframe['location'].replace(['University'], '2')
    dataframe['location'] = dataframe['location'].replace(['House'], '3')
    
    # timestamp
    if 'timestamp2' in dataframe.columns:
        dataframe['timestamp_text'] = dataframe['timestamp']
        dataframe['timestamp'] = dataframe['timestamp2']
        del dataframe['timestamp2']
    if 'outside_range' in dataframe.columns:
        del dataframe['outside_range']
    if 'time_sleeping' in dataframe.columns:
        del dataframe['time_sleeping']
    if 'wakeup_time' in dataframe.columns:
        del dataframe['wakeup_time']
    if 'sleep_time' in dataframe.columns:
        del dataframe['sleep_time']
    if 'distanceHome' in dataframe.columns:
        del dataframe['distanceHome']    
    if 'showActuation' in dataframe.columns:
        del dataframe['showActuation'] 
    #dataframe['timestamp'] = dataframe['timestamp'].astype('datetime64[ns]')
    #dataframe['timestamp'] = dataframe['timestamp'].astype('int64')
    #dataframe['timestamp'] = datetime.strptime(dataframe['timestamp'],'%Y-%m-%d %H:%M:%S').timestamp() * 1000
    #dataframe['timestamp'] = pd.to_datetime(dataframe['timestamp'],'%Y-%m-%d %H:%M:%S') * 1000
    #dataframe['timestamp'] = dataframe['timestamp'].dt.second()
    # data_type
    #dataframe['activity'] = dataframe['activity'].astype(np.integer) 
    #dataframe['class'] = dataframe['class'].astype(np.integer) 
    #dataframe['day_of_week'] = dataframe['day_of_week'].astype(np.integer) 
    #dataframe['phone_lock'] = dataframe['phone_lock'].astype(np.integer) 
    #dataframe['location'] = dataframe['location'].astype(np.integer) 
    dataframe['sound'] = dataframe['sound'].apply(convert_to_decibels)
    dataframe['proximity'] = dataframe['proximity'].apply(treat_data_to_0_1)
    dataframe['activity'] = dataframe['activity'].astype('int32')
    dataframe['class'] = dataframe['class'].astype('int32')
    dataframe['day_of_week'] = dataframe['day_of_week'].astype('int32')
    dataframe['phone_lock'] = dataframe['phone_lock'].astype('int32') 
    dataframe['location'] = dataframe['location'].astype('int32')
    dataframe['minutes_day'] = dataframe['minutes_day'].astype('float32')
    dataframe['time_to_next_alarm'] = dataframe['time_to_next_alarm'].astype('float32')
    dataframe['sound'] = dataframe['sound'].astype('float32')
    dataframe['light'] = dataframe['light'].astype('float32')
    dataframe['proximity'] = dataframe['proximity'].astype('float32')
    dataframe['timestamp'] = dataframe['timestamp'].astype('float32')
    
    temp = dataframe['class'].copy()
    del dataframe['class']
    dataframe['class'] = temp
    #return dataframe
    return dataframe

In [3]:
def transform_normalize_data(df):
    result = df.copy()
    for feature_name in df.columns:
        if feature_name != "class" and feature_name != "timestamp_text":
            max_value = df[feature_name].max()
            if(feature_name == "timestamp"):
                max_value = 1924905600.0 #2030-12-31 00:00:00  
            if(feature_name == "day_of_week"):
                max_value = 7            
            if(feature_name == "sound"):
                max_value = 403 # overhal higher value            
            if(feature_name == "location"):
                max_value = 3
            if(feature_name == "activity"):
                max_value = 5
            if(feature_name == "minutes_day"):
                max_value = 1439 # 24 horas
            if(feature_name == "time_to_next_alarm"):
                max_value = 604800000.0 # 2.8 years in ms
            min_value = df[feature_name].min()
            print(feature_name+" "+str(min_value)+" "+str(max_value))
            if feature_name != "showActuation":
                result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
            if(max_value == 0 and min_value == 0):
                result[feature_name] = 0.0
    return result

In [4]:
def transform_data_type(dataset):
    # diminui o tamanho do tipo de dado (float32 é mais que o suficiente)
    dataset['activity'] = dataset['activity'].astype('float32')
    dataset['location'] = dataset['location'].astype('float32')
    dataset['timestamp'] = dataset['timestamp'].astype('float32')
    dataset['day_of_week'] = dataset['day_of_week'].astype('float32')
    dataset['light'] = dataset['light'].astype('float32')
    dataset['phone_lock'] = dataset['phone_lock'].astype('float32')
    dataset['proximity'] = dataset['proximity'].astype('float32')
    dataset['sound'] = dataset['sound'].astype('float32')
    dataset['time_to_next_alarm'] = dataset['time_to_next_alarm'].astype('float32')
    
    return dataset

In [5]:
def transform_output_to_numeric(dataframe):
    dataframe['class'] = dataframe['class'].replace(['awake'], '0')
    dataframe['class'] = dataframe['class'].replace(['asleep'], '1')
    
    dataframe['class'] = dataframe['class'].astype('int32')
    
    return dataframe

In [6]:
def transform_output_to_str(dataframe):
    dataframe['class'] = dataframe['class'].apply(str)
    #print(dataframe['class'])
    dataframe['class'] = dataframe['class'].replace(to_replace='0',value='awake')
    dataframe['class'] = dataframe['class'].replace(to_replace='1',value='asleep')
    
    return dataframe

# Load all data file

In [7]:
dataset = pd.read_csv(outputFolders+fileRaw)

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136286 entries, 0 to 136285
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            136286 non-null  object 
 1   location            136286 non-null  object 
 2   distanceHome        136286 non-null  int64  
 3   showActuation       136286 non-null  bool   
 4   timestamp           136286 non-null  object 
 5   day_of_week         136286 non-null  object 
 6   light               136286 non-null  int64  
 7   phone_lock          136286 non-null  object 
 8   proximity           136286 non-null  int64  
 9   sound               136286 non-null  int64  
 10  time_to_next_alarm  136286 non-null  float64
 11  class               136286 non-null  object 
 12  timestamp2          136286 non-null  int64  
 13  minutes_day         136286 non-null  int64  
 14  sleep_time          136286 non-null  int64  
 15  wakeup_time         136286 non-nul

In [8]:
dataset

,activity,location,distanceHome,showActuation,timestamp,day_of_week,light,phone_lock,proximity,sound,time_to_next_alarm,class,timestamp2,minutes_day,sleep_time,wakeup_time,time_sleeping
0,Unknown,House,0,False,2018-05-15 02:46:57,Monday,0,not locked,10,1968,548040000.0,asleep,1526348817,166,1200,240,28800
1,Still,House,0,False,2018-05-15 02:47:27,Monday,0,not locked,10,11654,547980000.0,asleep,1526348847,167,1200,240,28800
2,Still,House,0,False,2018-05-15 02:47:57,Monday,0,not locked,10,11654,547980000.0,asleep,1526348877,167,1200,240,28800
3,Still,House,0,False,2018-05-15 02:48:28,Monday,0,not locked,10,1778,547920000.0,asleep,1526348908,168,1200,240,28800
4,Still,House,0,False,2018-05-15 02:48:57,Monday,0,not locked,10,1778,547920000.0,asleep,1526348937,168,1200,240,28800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136281,Still,House,0,False,2018-06-13 12:14:37,Wednesday,8,locked,5,0,59999.0,awake,1528888477,734,0,0,0
136282,Still,House,0,False,2018-06-13 12:18:08,Wednesday,11,locked,5,0,420000.0,awake,1528888688,738,0,0,0
136283,Still,House,0,False,2018-06-13 12:19:08,Wednesday,11,locked,5,0,359999.0,awake,1528888748,739,0,0,0
136284,Still,House,0,False,2018-06-13 12:19:38,Wednesday,12,locked,5,0,359999.0,awake,1528888778,739,0,0,0


In [9]:
# AllData Numeric to classification
datasetNumericClassification = transform_data(dataset)

datasetNumericClassification.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136286 entries, 0 to 136285
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            136286 non-null  int32  
 1   location            136286 non-null  int32  
 2   timestamp           136286 non-null  float32
 3   day_of_week         136286 non-null  int32  
 4   light               136286 non-null  float32
 5   phone_lock          136286 non-null  int32  
 6   proximity           136286 non-null  float32
 7   sound               136286 non-null  float32
 8   time_to_next_alarm  136286 non-null  float32
 9   minutes_day         136286 non-null  float32
 10  timestamp_text      136286 non-null  object 
 11  class               136286 non-null  int32  
dtypes: float32(6), int32(5), object(1)
memory usage: 6.8+ MB


In [10]:
# save files
datasetNumericClassification.to_csv(outputFolders+"allData"+fileNumericClassification, sep=',', encoding='utf-8', index=False)

#datasetNumericClassification

In [11]:
# data to Regression
datasetNumeric = transform_output_to_numeric(datasetNumericClassification)

datasetNumeric.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136286 entries, 0 to 136285
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            136286 non-null  int32  
 1   location            136286 non-null  int32  
 2   timestamp           136286 non-null  float32
 3   day_of_week         136286 non-null  int32  
 4   light               136286 non-null  float32
 5   phone_lock          136286 non-null  int32  
 6   proximity           136286 non-null  float32
 7   sound               136286 non-null  float32
 8   time_to_next_alarm  136286 non-null  float32
 9   minutes_day         136286 non-null  float32
 10  timestamp_text      136286 non-null  object 
 11  class               136286 non-null  int32  
dtypes: float32(6), int32(5), object(1)
memory usage: 6.8+ MB


In [12]:
# save files
datasetNumeric.to_csv(outputFolders+"allData"+fileNumeric, sep=',', encoding='utf-8', index=False)

#datasetNumeric

In [13]:
# normalization to Regression
datasetNumericNormalized = transform_normalize_data(datasetNumeric)

datasetNumericNormalized.info()

activity 1 5
location 1 3
timestamp 1526090900.0 1924905600.0
day_of_week 1 7
light 0.0 141769.0
phone_lock 0 1
proximity 0.0 1.0
sound 0.0 403
time_to_next_alarm -1.0 604800000.0
minutes_day 0.0 1439
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136286 entries, 0 to 136285
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            136286 non-null  float64
 1   location            136286 non-null  float64
 2   timestamp           136286 non-null  float32
 3   day_of_week         136286 non-null  float64
 4   light               136286 non-null  float32
 5   phone_lock          136286 non-null  float64
 6   proximity           136286 non-null  float32
 7   sound               136286 non-null  float32
 8   time_to_next_alarm  136286 non-null  float32
 9   minutes_day         136286 non-null  float32
 10  timestamp_text      136286 non-null  object 
 11  class               136286 non-nu

In [14]:
# save files
datasetNumericNormalized.to_csv(outputFolders+"allData"+fileNumericNormalized, sep=',', encoding='utf-8', index=False)

#datasetNumericNormalized

In [15]:
# transform to classification and print
datasetNumericClassificationNormalized = transform_output_to_str(datasetNumericNormalized)

datasetNumericClassificationNormalized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136286 entries, 0 to 136285
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            136286 non-null  float64
 1   location            136286 non-null  float64
 2   timestamp           136286 non-null  float32
 3   day_of_week         136286 non-null  float64
 4   light               136286 non-null  float32
 5   phone_lock          136286 non-null  float64
 6   proximity           136286 non-null  float32
 7   sound               136286 non-null  float32
 8   time_to_next_alarm  136286 non-null  float32
 9   minutes_day         136286 non-null  float32
 10  timestamp_text      136286 non-null  object 
 11  class               136286 non-null  object 
dtypes: float32(6), float64(4), object(2)
memory usage: 9.4+ MB


In [16]:
# save files
datasetNumericClassificationNormalized.to_csv(outputFolders+"allData"+fileNumericClassificationNormalized, sep=',', encoding='utf-8', index=False)

#datasetNumericClassificationNormalized

# Files by students

In [17]:
# Dataframe of all datasets
# initialize list of lists
dsTest   = [['PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc'], 
            ['rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw'], 
            ['RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI'], 
            ['VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is'], 
            ['Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw'], 
            ['XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA'], 
            ['YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw'], 
            ['ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM'], 
            ['ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']]

# Create the pandas DataFrame
folders = pd.DataFrame(dsTest, columns=['folder'])

#folders

In [18]:
for indexFolder, folder in folders.iterrows():
    print()
    dataset = pd.read_csv(outputFolders+"student_"+folder['folder']+".csv")
    # basic transformations 
    datasetNumericClassification = transform_data(dataset)
    #datasetNumeric = transform_output_to_numeric(datasetNumericClassification)
    #datasetNumeric = transform_data_type(datasetNumeric)
    # numeric used by the algorithm -> prediction
    datasetNumericNormalized = transform_data_type(datasetNumericClassification)
    datasetNumericNormalized = transform_normalize_data(datasetNumericNormalized)
    datasetNumericNormalized.to_csv(outputFolders+"student_"+folder['folder']+"_numeric.csv", sep=',', encoding='utf-8', index=False)
    # numeric used by the algorithm -> classification
    datasetNumericClassificationNormalized = transform_output_to_str(datasetNumericNormalized)
    datasetNumericClassificationNormalized.to_csv(outputFolders+"student_"+folder['folder']+"_nominal.csv", sep=',', encoding='utf-8', index=False)


activity 1.0 5
location 1.0 3
timestamp 1526348800.0 1924905600.0
day_of_week 1.0 7
light 0.0 10243.0
phone_lock 0.0 1.0
proximity 0.0 1.0
sound 0.0 403
time_to_next_alarm 0.0 604800000.0
minutes_day 0.0 1439

activity 1.0 5
location 1.0 3
timestamp 1526310900.0 1924905600.0
day_of_week 1.0 7
light 0.0 141769.0
phone_lock 0.0 1.0
proximity 0.0 1.0
sound 0.0 403
time_to_next_alarm -1.0 604800000.0
minutes_day 0.0 1439

activity 1.0 5
location 1.0 3
timestamp 1526090900.0 1924905600.0
day_of_week 1.0 7
light 0.0 65535.0
phone_lock 0.0 1.0
proximity 0.0 1.0
sound 0.0 403
time_to_next_alarm 0.0 604800000.0
minutes_day 0.0 1439

activity 1.0 5
location 1.0 3
timestamp 1526144800.0 1924905600.0
day_of_week 1.0 7
light 0.0 0.0
phone_lock 0.0 1.0
proximity 0.0 1.0
sound 0.0 403
time_to_next_alarm -1.0 604800000.0
minutes_day 0.0 1439

activity 1.0 5
location 1.0 3
timestamp 1526314500.0 1924905600.0
day_of_week 1.0 7
light 0.0 112596.0
phone_lock 0.0 1.0
proximity 0.0 1.0
sound 0.0 403
time_t